# MLP with FrenchC2C Dataset, FTU - mk6

## Load Dependencies <a id="libraries"></a>

In [1]:
import pandas as pd
import numpy as np
import timeit
import os
import math

In [2]:
import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
import warnings
warnings.filterwarnings('ignore')

### Notebook options and parameters

In [5]:
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250

In [6]:
# random seed for consistency in pytorch and numpy
SEED = 23

torch.manual_seed(SEED)
np.random.seed(SEED)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Hyperparameter options

In [8]:
# activation functions
elu = nn.ELU()
leaky = nn.LeakyReLU()
relu = nn.ReLU()
tanh = nn.Tanh()
sigmoid = nn.Sigmoid()

# loss functions
mse = nn.MSELoss()
bce = nn.BCELoss()
kld = nn.KLDivLoss()

# optimizers
adam = torch.optim.Adam
sgd = torch.optim.SGD

## Custom Functions <a id="functions"></a>

#### Create dataset for DataLoader

In [9]:
class dataset(Dataset):
    def __init__(self, data, target):
        self.X = torch.tensor(data, dtype=torch.float32, device=device)
        self.y = torch.tensor(target, dtype=torch.float32, device=device)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

#### Training function

In [10]:
def train(data, model, criterion, metric, optimizer):
    size = len(data.dataset)
    batches = len(data)
    running_loss = []
    acc = 0.0
    correct = 0
    predictions = []
    actuals = []
    loss = 0.0
    acc = 0.0
    current = 0
    
    for batch, (X, y) in enumerate(data,1):
        
        # forward step
        optimizer.zero_grad() # zero out the gradients
        y_hat = model(X)  #forward pass
        
        y = y.unsqueeze(-1) # reduce dimensions of tensor from [256, 1] to [256]
        
        loss = criterion(y_hat, y)  # calculate loss
       
        #  backprop step
        loss.backward()       # backward pass through model; computes gradients
        optimizer.step()      # update weights
        
        # metrics
        y = y.int()
        acc = metric(y_hat, y)

        y_hat = y_hat.cpu().detach().numpy()
        y_hat = y_hat.reshape(-1)
        
        y = y.cpu().detach().numpy()
        y = y.reshape(-1)
        
        predictions.append(y_hat)
        actuals.append(y)
            
        running_loss.append(loss.item())
    
        ## every 50 batches, store the results
        #if batch % 50 == 0:
        #    current = batch * len(X)
        
    print(f"\tTraining\tAccuracy: {acc:1.10f}\tLoss: {loss:1.10f}")
    
    return predictions, running_loss

#### Testing function

In [11]:
def test(data, model, criterion, metric):
    size = len(data.dataset)
    batches = len(data)
    tst_loss = 0
    correct = 0
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X, y in data:
            y_hat = model(X)
            y = y.unsqueeze(-1)
            tst_loss += criterion(y_hat, y).item()
            
            y = y.int()
            acc = metric(y_hat, y)
            #acc = acc.cpu().detach().numpy()
            
            y_hat = y_hat.cpu().detach().numpy()
            y_hat = y_hat.reshape(-1)
        
            y = y.cpu().detach().numpy()
            y = y.reshape(-1)
        
            predictions.append(y_hat)
            actuals.append(y)
            
    tst_loss /= batches
    correct /= size
    
    print(f"\tTest error\tAccuracy: {acc:1.10f}\tLoss: {tst_loss:1.10f}")
    return predictions, tst_loss, acc

#### Evaluation - train and test

In [12]:
def evaluate(training_data, test_data, model, loss_function, metric, optimizer, epochs, early_stop=True):
    training_results = []
    training_actuals = []
    training_losses = []

    testing_results = []
    testing_actuals = []

    start = timeit.default_timer()
    
    # early stop criteria
    
    default_patience = 3
    patience = default_patience
    running_loss = []
    running_acc = []

    for epoch in range(1,epochs+1):
        print (f"Epoch {epoch} / {epochs}")

        trng_result, trng_loss = train(training_data, model, loss_function, metric, optimizer)
        tst_result, tst_loss, acc = test(test_data, model, loss_function, metric)
        
        training_results.append(trng_result)
        training_losses.append(trng_loss)
        testing_results.append(tst_result)
        running_loss.append(tst_loss)
        running_acc.append(acc)
        
        # check for early stop
        if epoch > 3:            
            if early_stop and ((tst_loss >= running_loss[-2]) or math.isclose(acc,running_acc[-2], abs_tol=1e-8)):
                patience = patience - 1
            elif early_stop and (tst_loss < running_loss[-2]):
                patience = default_patience

            if early_stop and (patience <= 0):
                print("Early stop - ", end="")
                break

    
    print("Finished")
    print("* " * 30)
    stop = timeit.default_timer()
    print(f"Execution time (in seconds): {stop - start}")

    return training_results, training_losses, testing_results

#### Multilayer Perception (MLP)

In [13]:
class MLP(nn.Module):
    
    def __init__(self, in_size, hidden1=relu, hidden2=relu, hidden3=relu, out_layer=sigmoid):
        super(MLP, self).__init__()
        
        # construct model dependent on size of inputs
        out1_2 = 2*in_size // 3
        out2_3 = 2*out1_2 // 3
        out3_4 = 2*out2_3 // 3

        self.linear_stack = nn.Sequential(
            nn.Linear(in_size, out1_2),
            hidden1,
            nn.Linear(out1_2, out2_3),
            hidden2,
            nn.Linear(out2_3, out3_4),
            hidden3,
            nn.Linear(out3_4, 1),
            out_layer
        )
        
    def forward(self, x):
        out = self.linear_stack(x)

        return out

#### Flatten function

In [14]:
def flatten(thing):
    '''
    thing: a list
    
    flatten receives a multi-level list and flatten it down 2 layers
    '''
    #thing = [element for sublist in thing for element in sublist]
    return [element for sublist in thing for element in sublist]

#### Prepare data for eval and analysis

In [15]:
def prep(name, training_results, testing_results):
    trng_preds = [[] for _ in range(len(training_results))]

    tst_preds = [[] for _ in range(len(testing_results))]
    
    for i in range(len(training_results)):
        trng_preds[i] = flatten(training_results[i])

        tst_preds[i] = flatten(testing_results[i])

    return trng_preds, tst_preds

#### Send results to tensorboard

In [16]:
def to_tensorboard(tailend, directory, dataset, train_preds, test_preds, y_train, y_test):
    acc_trng_relu = []
    acc_tst_relu = []
    loss_relu = []

    writer = SummaryWriter(log_dir=directory)

    for i in range(len(train_preds)):
        y_hat = np.array(np.round(train_preds[i]))
        y_hat_tst = np.array(np.round(test_preds[i]))
        trng_acc = (y_hat == y_train).sum()/len(train_preds[i])
        test_acc = (y_hat_tst == y_test).sum()/len(test_preds[i])

        avg_loss = np.average(training_losses[i])

        acc_trng_relu.append(trng_acc)
        acc_tst_relu.append(test_acc)
        loss_relu.append(avg_loss)
        writer.add_scalar(f"Training Accuracy {tailend}", trng_acc, i)
        writer.add_scalar(f"Test Accuracy {tailend}", test_acc, i)
        writer.add_scalar(f"Training Loss {tailend}", avg_loss, i)
        writer.add_pr_curve(f"Precision-Recall Curve {tailend}", y_test, y_hat_tst, i)

    t, _ = next(iter(dataset))
    writer.add_graph(model, t)

    writer.close()

#### Save model

In [17]:
def custom_save(name, data, kind=1):
    '''
    name : string
        designated filename
    data : data or pytorch model
        the data to save
    kind : int
        sentinel value - 1 if pytorch model, 0 otherwise
    
    custom_save stores the data passed into the function into a file with the provided name
    '''
    
    if kind == 1:
        ex = ".pth"
    else:
        ex = ".parquet"
    
    sentinel = True
    i = 1

    while sentinel:
        dirlist = os.listdir()

        if name not in dirlist:
            if kind == 1:
                torch.save(data, name)
            else:
                data.to_parquet(name)
            print(f"{name} has been saved.")                
            sentinel = False
        if name in dirlist:
            print(f"{name} already exists.", end=" ")
            temp, ext = name.split(ex)
            if "_v" in temp:
                temp, _ = temp.split("_v")
            name = f"{temp}_v{i}{ex}"
            i = i + 1
            print(f"Changing file name to: {name}")

# Import Data<a id="data"></a>
##### French C2C dataset

#### Uncomment next when using google colab

In [18]:
#data = pd.read_parquet("https://github.com/ollin23/bias/blob/main/french_mk2.parquet?raw=true")

#### Use the following cell if the data is stored locally

In [19]:
data = pd.read_parquet("french_mk2.parquet")

#### Peak the data

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98913 entries, 0 to 98912
Columns: 224 entries, socialNbFollowers to language_it
dtypes: bool(4), float64(2), int64(9), uint8(209)
memory usage: 28.4 MB


In [21]:
data.shape

(98913, 224)

In [22]:
data.head()

,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,productsPassRate,productsWished,productsBought,hasAnyApp,hasAndroidApp,hasIosApp,hasProfilePicture,daysSinceLastLogin,seniorityAsMonths,target,civGen_1,civGen_2,civGen_3,gender_F,gender_M,countryCode_ad,countryCode_ae,countryCode_af,countryCode_ag,countryCode_ai,countryCode_al,countryCode_am,countryCode_an,countryCode_ao,countryCode_aq,countryCode_ar,countryCode_as,countryCode_at,countryCode_au,countryCode_aw,countryCode_az,countryCode_ba,countryCode_bb,countryCode_bd,countryCode_be,countryCode_bf,countryCode_bg,countryCode_bh,countryCode_bj,countryCode_bl,countryCode_bm,countryCode_bn,countryCode_bo,countryCode_br,countryCode_bs,countryCode_bv,countryCode_bw,countryCode_by,countryCode_bz,countryCode_ca,countryCode_cd,countryCode_cf,countryCode_cg,countryCode_ch,countryCode_ci,countryCode_cl,countryCode_cm,countryCode_cn,countryCode_co,countryCode_cr,countryCode_cu,countryCode_cy,countryCode_cz,countryCode_de,countryCode_dj,countryCode_dk,countryCode_dm,countryCode_do,countryCode_dz,countryCode_ec,countryCode_ee,countryCode_eg,countryCode_er,countryCode_es,countryCode_et,countryCode_fi,countryCode_fj,countryCode_fk,countryCode_fo,countryCode_fr,countryCode_ga,countryCode_gb,countryCode_gd,countryCode_ge,countryCode_gg,countryCode_gh,countryCode_gi,countryCode_gm,countryCode_gn,countryCode_gp,countryCode_gr,countryCode_gs,countryCode_gt,countryCode_gu,countryCode_gy,countryCode_hk,countryCode_hn,countryCode_hr,countryCode_ht,countryCode_hu,countryCode_ic,countryCode_id,countryCode_ie,countryCode_il,countryCode_im,countryCode_in,countryCode_iq,countryCode_ir,countryCode_is,countryCode_it,countryCode_je,countryCode_jm,countryCode_jo,countryCode_jp,countryCode_ke,countryCode_kg,countryCode_kh,countryCode_ki,countryCode_kn,countryCode_kp,countryCode_kr,countryCode_kw,countryCode_ky,countryCode_kz,countryCode_la,countryCode_lb,countryCode_lc,countryCode_li,countryCode_lk,countryCode_lt,countryCode_lu,countryCode_lv,countryCode_ly,countryCode_ma,countryCode_mc,countryCode_md,countryCode_mg,countryCode_mk,countryCode_ml,countryCode_mm,countryCode_mn,countryCode_mo,countryCode_mq,countryCode_mr,countryCode_mt,countryCode_mu,countryCode_mv,countryCode_mx,countryCode_my,countryCode_na,countryCode_nc,countryCode_ne,countryCode_nf,countryCode_ng,countryCode_ni,countryCode_nl,countryCode_no,countryCode_np,countryCode_nz,countryCode_om,countryCode_pa,countryCode_pe,countryCode_pf,countryCode_ph,countryCode_pk,countryCode_pl,countryCode_pr,countryCode_pt,countryCode_pw,countryCode_py,countryCode_qa,countryCode_re,countryCode_ro,countryCode_rs,countryCode_ru,countryCode_rw,countryCode_sa,countryCode_se,countryCode_sg,countryCode_si,countryCode_sj,countryCode_sk,countryCode_sn,countryCode_sr,countryCode_sv,countryCode_sy,countryCode_sz,countryCode_tc,countryCode_td,countryCode_tg,countryCode_th,countryCode_tj,countryCode_tn,countryCode_tr,countryCode_tt,countryCode_tw,countryCode_tz,countryCode_ua,countryCode_ug,countryCode_um,countryCode_us,countryCode_uy,countryCode_uz,countryCode_vc,countryCode_ve,countryCode_vg,countryCode_vi,countryCode_vn,countryCode_vu,countryCode_ws,countryCode_yt,countryCode_za,countryCode_zm,countryCode_zw,language_de,language_en,language_es,language_fr,language_it
0,147,10,77,26,174,74.0,104,1,True,False,True,True,11,106.53,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,167,8,2,19,170,99.0,0,0,True,False,True,True,12,106.80,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
list(data.columns)

['socialNbFollowers',
 'socialNbFollows',
 'socialProductsLiked',
 'productsListed',
 'productsSold',
 'productsPassRate',
 'productsWished',
 'productsBought',
 'hasAnyApp',
 'hasAndroidApp',
 'hasIosApp',
 'hasProfilePicture',
 'daysSinceLastLogin',
 'seniorityAsMonths',
 'target',
 'civGen_1',
 'civGen_2',
 'civGen_3',
 'gender_F',
 'gender_M',
 'countryCode_ad',
 'countryCode_ae',
 'countryCode_af',
 'countryCode_ag',
 'countryCode_ai',
 'countryCode_al',
 'countryCode_am',
 'countryCode_an',
 'countryCode_ao',
 'countryCode_aq',
 'countryCode_ar',
 'countryCode_as',
 'countryCode_at',
 'countryCode_au',
 'countryCode_aw',
 'countryCode_az',
 'countryCode_ba',
 'countryCode_bb',
 'countryCode_bd',
 'countryCode_be',
 'countryCode_bf',
 'countryCode_bg',
 'countryCode_bh',
 'countryCode_bj',
 'countryCode_bl',
 'countryCode_bm',
 'countryCode_bn',
 'countryCode_bo',
 'countryCode_br',
 'countryCode_bs',
 'countryCode_bv',
 'countryCode_bw',
 'countryCode_by',
 'countryCode_bz',
 'co

### Process Data for Neural Net Usage <a id="convert"></a>

#### Drop unnecessary features

In [24]:
data = data.drop(["gender_M"], axis=1)

#### Rename some features

In [25]:
new_names = {'gender_F': 'gender'}

In [26]:
data.rename(new_names, axis=1, inplace=True)

#### Split features into X and y

In [27]:
X = data.drop("target", axis=1).values.astype(np.float32)
y = data.target.values.astype(np.float32)

#### Standardize data

In [28]:
scaler = StandardScaler()
scaledX = scaler.fit_transform(X)

#### Split into training and testing sets

In [29]:
X_train, X_test, y_train, y_test = train_test_split(scaledX, y, test_size=0.3,
                                                        random_state=SEED, stratify=y)

In [30]:
X_train.shape

(69239, 222)

In [31]:
X_test.shape

(29674, 222)

In [32]:
col = list(data.columns).index("gender")
col

18

gender = X_test[:,18]

In [34]:
temp = np.round(scaler.inverse_transform(X_test)).astype(int)

X_train = np.delete(X_train, col, axis=1)
X_test = np.delete(X_test, col, axis=1)

#### Make datasets

In [35]:
scaled_train = dataset(X_train, y_train)
scaled_test = dataset(X_test, y_test)

##### Construct test results dataframe for persistent storage

In [36]:
features = list(data.drop("target", axis=1).columns)

df = pd.DataFrame(temp, columns=features)
gender = df.gender.values

# relocate gender feature in dataframe for later human readability
df.drop("gender", axis=1, inplace=True)
df["gender"] = gender
df["target"] = y_test.astype(int)

In [37]:
df.head()

,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,productsPassRate,productsWished,productsBought,hasAnyApp,hasAndroidApp,hasIosApp,hasProfilePicture,daysSinceLastLogin,seniorityAsMonths,civGen_1,civGen_2,civGen_3,countryCode_ad,countryCode_ae,countryCode_af,countryCode_ag,countryCode_ai,countryCode_al,countryCode_am,countryCode_an,countryCode_ao,countryCode_aq,countryCode_ar,countryCode_as,countryCode_at,countryCode_au,countryCode_aw,countryCode_az,countryCode_ba,countryCode_bb,countryCode_bd,countryCode_be,countryCode_bf,countryCode_bg,countryCode_bh,countryCode_bj,countryCode_bl,countryCode_bm,countryCode_bn,countryCode_bo,countryCode_br,countryCode_bs,countryCode_bv,countryCode_bw,countryCode_by,countryCode_bz,countryCode_ca,countryCode_cd,countryCode_cf,countryCode_cg,countryCode_ch,countryCode_ci,countryCode_cl,countryCode_cm,countryCode_cn,countryCode_co,countryCode_cr,countryCode_cu,countryCode_cy,countryCode_cz,countryCode_de,countryCode_dj,countryCode_dk,countryCode_dm,countryCode_do,countryCode_dz,countryCode_ec,countryCode_ee,countryCode_eg,countryCode_er,countryCode_es,countryCode_et,countryCode_fi,countryCode_fj,countryCode_fk,countryCode_fo,countryCode_fr,countryCode_ga,countryCode_gb,countryCode_gd,countryCode_ge,countryCode_gg,countryCode_gh,countryCode_gi,countryCode_gm,countryCode_gn,countryCode_gp,countryCode_gr,countryCode_gs,countryCode_gt,countryCode_gu,countryCode_gy,countryCode_hk,countryCode_hn,countryCode_hr,countryCode_ht,countryCode_hu,countryCode_ic,countryCode_id,countryCode_ie,countryCode_il,countryCode_im,countryCode_in,countryCode_iq,countryCode_ir,countryCode_is,countryCode_it,countryCode_je,countryCode_jm,countryCode_jo,countryCode_jp,countryCode_ke,countryCode_kg,countryCode_kh,countryCode_ki,countryCode_kn,countryCode_kp,countryCode_kr,countryCode_kw,countryCode_ky,countryCode_kz,countryCode_la,countryCode_lb,countryCode_lc,countryCode_li,countryCode_lk,countryCode_lt,countryCode_lu,countryCode_lv,countryCode_ly,countryCode_ma,countryCode_mc,countryCode_md,countryCode_mg,countryCode_mk,countryCode_ml,countryCode_mm,countryCode_mn,countryCode_mo,countryCode_mq,countryCode_mr,countryCode_mt,countryCode_mu,countryCode_mv,countryCode_mx,countryCode_my,countryCode_na,countryCode_nc,countryCode_ne,countryCode_nf,countryCode_ng,countryCode_ni,countryCode_nl,countryCode_no,countryCode_np,countryCode_nz,countryCode_om,countryCode_pa,countryCode_pe,countryCode_pf,countryCode_ph,countryCode_pk,countryCode_pl,countryCode_pr,countryCode_pt,countryCode_pw,countryCode_py,countryCode_qa,countryCode_re,countryCode_ro,countryCode_rs,countryCode_ru,countryCode_rw,countryCode_sa,countryCode_se,countryCode_sg,countryCode_si,countryCode_sj,countryCode_sk,countryCode_sn,countryCode_sr,countryCode_sv,countryCode_sy,countryCode_sz,countryCode_tc,countryCode_td,countryCode_tg,countryCode_th,countryCode_tj,countryCode_tn,countryCode_tr,countryCode_tt,countryCode_tw,countryCode_tz,countryCode_ua,countryCode_ug,countryCode_um,countryCode_us,countryCode_uy,countryCode_uz,countryCode_vc,countryCode_ve,countryCode_vg,countryCode_vi,countryCode_vn,countryCode_vu,countryCode_ws,countryCode_yt,countryCode_za,countryCode_zm,countryCode_zw,language_de,language_en,language_es,language_fr,language_it,gender,target
0,3,8,0,0,0,0,0,0,0,0,0,1,699,106,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,3,8,41,0,0,0,0,0,1,0,1,1,605,107,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

##### Note
For this dataset: Female == 1, Male == 0

# Experiment Phase 1 - French C2C Data <a id="experiments"></a>

## Trial 1 - ReLU (Loans) <a id="relu"></a>

### Preliminaries

In [38]:
suffix = "mk6_FTU"
directory = f"FrenchC2C_{suffix}"
prefix = "french"

#### Hyperparameters <a id="hyperparameters"></a>

In [39]:
alpha = 1e-5 # 0.00001
epochs = 150
batches = 100
hidden1 = relu
hidden2 = relu
hidden3 = relu
out_layer = sigmoid

#### DataLoader

In [40]:
# Helper function to feed data into pytorch neural network, only needs to execute once
training_s = DataLoader(scaled_train, batch_size=batches)
test_s = DataLoader(scaled_test, batch_size=batches)

#### Construct Model

In [41]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=221, out_features=147, bias=True)
    (1): ReLU()
    (2): Linear(in_features=147, out_features=98, bias=True)
    (3): ReLU()
    (4): Linear(in_features=98, out_features=65, bias=True)
    (5): ReLU()
    (6): Linear(in_features=65, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [42]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs)

Epoch 1 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.1781933606
	Test error	Accuracy: 0.9729729891	Loss: 0.1718017363
Epoch 2 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0979202613
	Test error	Accuracy: 0.9729729891	Loss: 0.0838969486
Epoch 3 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0729199052
	Test error	Accuracy: 0.9729729891	Loss: 0.0539153829
Epoch 4 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0697045550
	Test error	Accuracy: 0.9729729891	Loss: 0.0488878967
Epoch 5 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0686862990
	Test error	Accuracy: 0.9729729891	Loss: 0.0468340105
Epoch 6 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0677005574
	Test error	Accuracy: 0.9729729891	Loss: 0.0450122785
Early stop - Finished
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
Execution time (in seconds): 11.247413384000538


### Prepare results for analysis

In [43]:
tailend = f"ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save ReLU Metrics to Tensorboard <a id="relu-metrics-for-tensorboard"></a>

In [44]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ReLU predictions to persistent storage

In [45]:
df["pred_relu"] = np.array(np.round(test_preds[-1])).astype(int)

name = f"{prefix}_relu_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_relu_scaled.pth")

french_relu_mk6_FTU.pth has been saved.


## Trial 2 - Tanh (French) <a id="tanh"></a>

#### Hyperparameters

In [46]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = tanh
hidden2 = tanh
hidden3 = tanh
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [47]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=221, out_features=147, bias=True)
    (1): Tanh()
    (2): Linear(in_features=147, out_features=98, bias=True)
    (3): Tanh()
    (4): Linear(in_features=98, out_features=65, bias=True)
    (5): Tanh()
    (6): Linear(in_features=65, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Training and test

In [48]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs)

Epoch 1 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.1624379605
	Test error	Accuracy: 0.9324324131	Loss: 0.1621053060
Epoch 2 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0840269253
	Test error	Accuracy: 0.9324324131	Loss: 0.0771046731
Epoch 3 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0642277673
	Test error	Accuracy: 0.9459459186	Loss: 0.0492941563
Epoch 4 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0603932850
	Test error	Accuracy: 0.9459459186	Loss: 0.0411213714
Epoch 5 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0588765219
	Test error	Accuracy: 0.9459459186	Loss: 0.0371625597
Epoch 6 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0574436672
	Test error	Accuracy: 0.9594594836	Loss: 0.0344964960
Epoch 7 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0554480553
	Test error	Accuracy: 0.9594594836	Loss: 0.0323231756
Epoch 8 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0524003692
	Test error	Accuracy: 0.9594594836	Loss: 0.0302766738
Epoch 9 / 150
	Training	Accuracy: 0.9230

#### Prepare tanh results for analysis

In [49]:
tailend = f"Tanh ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Generate tensorboard summary

In [50]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Store Tanh results in dataframe 

In [51]:
df["pred_tanh"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_tanh_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_tanh_scaled.pth")

french_tanh_mk6_FTU.pth has been saved.


## Trial 3 - ELU (Loans)<a id="elu"></a>

#### Hyperparameters

In [52]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = elu
hidden2 = elu
hidden3 = elu
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [53]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=221, out_features=147, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=147, out_features=98, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=98, out_features=65, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=65, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [54]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs)

Epoch 1 / 150
	Training	Accuracy: 0.9487179518	Loss: 0.1972700059
	Test error	Accuracy: 0.9594594836	Loss: 0.1917170285
Epoch 2 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0939334258
	Test error	Accuracy: 0.9729729891	Loss: 0.0841748872
Epoch 3 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0644536093
	Test error	Accuracy: 0.9729729891	Loss: 0.0463492838
Epoch 4 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0603492633
	Test error	Accuracy: 0.9729729891	Loss: 0.0383174937
Epoch 5 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0588531569
	Test error	Accuracy: 0.9729729891	Loss: 0.0351410354
Epoch 6 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0573296063
	Test error	Accuracy: 0.9729729891	Loss: 0.0330403169
Early stop - Finished
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
Execution time (in seconds): 11.306373033992713


#### Prepare ELU results for analysis

In [55]:
tailend = f"ELU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save metrics to tensorboad

In [56]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ELU test results to persistent storage

In [57]:
df["pred_elu"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_elu_{suffix}.pth"
custom_save(name, model, 1)
#torch.save(model, f"{prefix}_elu_scaled.pth")

french_elu_mk6_FTU.pth has been saved.


## Trial 4 - Leaky ReLU (Loans) <a id="leaky"></a>

#### Hyperparameters

In [58]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = leaky
hidden2 = leaky
hidden3 = leaky
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct Model

In [59]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=221, out_features=147, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=147, out_features=98, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=98, out_features=65, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=65, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [60]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs)

Epoch 1 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.2220799923
	Test error	Accuracy: 0.9729729891	Loss: 0.2156608556
Epoch 2 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.1128015742
	Test error	Accuracy: 0.9729729891	Loss: 0.0967751579
Epoch 3 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0748251826
	Test error	Accuracy: 0.9729729891	Loss: 0.0534706498
Epoch 4 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0714692250
	Test error	Accuracy: 0.9729729891	Loss: 0.0482412588
Epoch 5 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0706762448
	Test error	Accuracy: 0.9729729891	Loss: 0.0461898795
Epoch 6 / 150
	Training	Accuracy: 0.9230769277	Loss: 0.0700260773
	Test error	Accuracy: 0.9729729891	Loss: 0.0444693083
Early stop - Finished
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
Execution time (in seconds): 11.397027722996427


#### Prepare Leaky ReLU data for analysis

In [61]:
tailend = f"Leaky ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Add Leaky ReLU data to persistent storage

In [62]:
df["pred_leaky"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_leaky_{suffix}.pth"
custom_save(name, model,1)
# torch.save(model, "loans_leaky_scaled.pth")

french_leaky_mk6_FTU.pth has been saved.


# Results

In [63]:
df.head()

,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,productsPassRate,productsWished,productsBought,hasAnyApp,hasAndroidApp,hasIosApp,hasProfilePicture,daysSinceLastLogin,seniorityAsMonths,civGen_1,civGen_2,civGen_3,countryCode_ad,countryCode_ae,countryCode_af,countryCode_ag,countryCode_ai,countryCode_al,countryCode_am,countryCode_an,countryCode_ao,countryCode_aq,countryCode_ar,countryCode_as,countryCode_at,countryCode_au,countryCode_aw,countryCode_az,countryCode_ba,countryCode_bb,countryCode_bd,countryCode_be,countryCode_bf,countryCode_bg,countryCode_bh,countryCode_bj,countryCode_bl,countryCode_bm,countryCode_bn,countryCode_bo,countryCode_br,countryCode_bs,countryCode_bv,countryCode_bw,countryCode_by,countryCode_bz,countryCode_ca,countryCode_cd,countryCode_cf,countryCode_cg,countryCode_ch,countryCode_ci,countryCode_cl,countryCode_cm,countryCode_cn,countryCode_co,countryCode_cr,countryCode_cu,countryCode_cy,countryCode_cz,countryCode_de,countryCode_dj,countryCode_dk,countryCode_dm,countryCode_do,countryCode_dz,countryCode_ec,countryCode_ee,countryCode_eg,countryCode_er,countryCode_es,countryCode_et,countryCode_fi,countryCode_fj,countryCode_fk,countryCode_fo,countryCode_fr,countryCode_ga,countryCode_gb,countryCode_gd,countryCode_ge,countryCode_gg,countryCode_gh,countryCode_gi,countryCode_gm,countryCode_gn,countryCode_gp,countryCode_gr,countryCode_gs,countryCode_gt,countryCode_gu,countryCode_gy,countryCode_hk,countryCode_hn,countryCode_hr,countryCode_ht,countryCode_hu,countryCode_ic,countryCode_id,countryCode_ie,countryCode_il,countryCode_im,countryCode_in,countryCode_iq,countryCode_ir,countryCode_is,countryCode_it,countryCode_je,countryCode_jm,countryCode_jo,countryCode_jp,countryCode_ke,countryCode_kg,countryCode_kh,countryCode_ki,countryCode_kn,countryCode_kp,countryCode_kr,countryCode_kw,countryCode_ky,countryCode_kz,countryCode_la,countryCode_lb,countryCode_lc,countryCode_li,countryCode_lk,countryCode_lt,countryCode_lu,countryCode_lv,countryCode_ly,countryCode_ma,countryCode_mc,countryCode_md,countryCode_mg,countryCode_mk,countryCode_ml,countryCode_mm,countryCode_mn,countryCode_mo,countryCode_mq,countryCode_mr,countryCode_mt,countryCode_mu,countryCode_mv,countryCode_mx,countryCode_my,countryCode_na,countryCode_nc,countryCode_ne,countryCode_nf,countryCode_ng,countryCode_ni,countryCode_nl,countryCode_no,countryCode_np,countryCode_nz,countryCode_om,countryCode_pa,countryCode_pe,countryCode_pf,countryCode_ph,countryCode_pk,countryCode_pl,countryCode_pr,countryCode_pt,countryCode_pw,countryCode_py,countryCode_qa,countryCode_re,countryCode_ro,countryCode_rs,countryCode_ru,countryCode_rw,countryCode_sa,countryCode_se,countryCode_sg,countryCode_si,countryCode_sj,countryCode_sk,countryCode_sn,countryCode_sr,countryCode_sv,countryCode_sy,countryCode_sz,countryCode_tc,countryCode_td,countryCode_tg,countryCode_th,countryCode_tj,countryCode_tn,countryCode_tr,countryCode_tt,countryCode_tw,countryCode_tz,countryCode_ua,countryCode_ug,countryCode_um,countryCode_us,countryCode_uy,countryCode_uz,countryCode_vc,countryCode_ve,countryCode_vg,countryCode_vi,countryCode_vn,countryCode_vu,countryCode_ws,countryCode_yt,countryCode_za,countryCode_zm,countryCode_zw,language_de,language_en,language_es,language_fr,language_it,gender,target,pred_relu,pred_tanh,pred_elu,pred_leaky
0,3,8,0,0,0,0,0,0,0,0,0,1,699,106,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1,3,8,41,0,0,0,0,0,1,0,1,1,605,107,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Save data

In [64]:
name = f"{prefix}_results_{suffix}.parquet"
custom_save(name, df, 0)

french_results_mk6_FTU.parquet has been saved.
